In [ ]:
%matplotlib inline
from pprint import pprint
from decimal import Decimal
import itertools
import metrics
import pandas

import simulate
import harvesting
import market
import withdrawal
import numpy

In [ ]:
import seaborn
from matplotlib import pyplot as plt
import matplotlib
seaborn.set(style="whitegrid")
seaborn.set_context('poster')

In [ ]:
series = market.Returns_US_1871()

In [ ]:
def mk_lens(type_):
    m = {'Withdrawal (Nominal)': lambda x: float(x.withdraw_n),
         'Withdrawal (Real)' : lambda x: float(x.withdraw_r),
         'Portfolio (Nominal)': lambda x: float(x.portfolio_pre.value_n),
         'Portfolio (Real)': lambda x: float(x.portfolio_pre.value_r),
         'Bond %': lambda x: float(x.portfolio_pre.bonds / (x.portfolio_pre.bonds + x.portfolio_pre.stocks))
    }
    lens = m[type_]
    lens.__label__ = type_
    return lens

def fn(lens, x):
    return [lens(_) for _ in x]

In [ ]:
def run_sim(year, key, length=35):
    h = {
        'tent': lambda p: harvesting.ParameterGlidepath(p, 0.4, 0.75, 30),
        'static-60': harvesting.make_rebalancer(0.60),
    }

    portfolio = (400_000, 600_000)
    
    return simulate.withdrawals(series.iter_from(year),
                                 withdraw=lambda p, s: withdrawal.VPW(p, s, years_left=40),
                                 years=length,
                                 portfolio=portfolio,
                                 harvesting=h[key])

In [ ]:
def cew_one(year):
    tent = run_sim(year, 'tent')
    static = run_sim(year, 'static-60')
    
    df = pandas.DataFrame({
        'tent' : [x.withdraw_r for x in tent],
        'static-60' : [x.withdraw_r for x in static],
    })
    return df.apply(metrics.cew)

def cew_all():
    df = pandas.DataFrame(index=range(1887, 2019 - 30 + 1), columns=['tent', 'static-60'])
    for i in range(1887, 2019 - 30 + 1):
        r = cew_one(i)
        df.loc[i] = r
    return df

In [ ]:
cew_one(1929)

In [ ]:
%time
df = cew_all()

In [ ]:
df.head()

In [ ]:
tent_wins = df[df['tent'] > df['static-60']]
print(len(tent_wins))
diff = df['tent'] - df['static-60']
plt.figure(figsize=(8,6))
plt.title('Tent CEW - 60/40 CEW (difference)')
seaborn.lineplot(data=diff.astype(float))

In [ ]:
diff[diff > 0]

In [ ]:
tent_loses = df[df['static-60'] > df['tent']]
print('Chance of tent wins', len(tent_wins) / (len(tent_wins) + len(tent_loses)))
print(diff[diff>0].median())
print(diff[diff<0].median())

In [ ]:
def chart_one(year, lens):
    tent = run_sim(year, 'tent')
    static = run_sim(year, 'static-60')
    
    df = pandas.DataFrame({
        'tent' : fn(lens, tent),
        'static-60' : fn(lens, static),
    })
    g = seaborn.relplot(
            data=df,
            kind='line',
            aspect=2,
        )
    g.fig.autofmt_xdate()
    g.despine(left=True, bottom=True, offset=20)
    g.fig.suptitle(f'Retirement in {year}')
    return g

In [ ]:
chart_one(1966, mk_lens('Portfolio (Real)'))
#chart_one(1929, mk_lens('Bond %'))
s = df.loc[1929]

In [ ]:
def one(year, lens):
    tent = run_sim(year, 'tent')
    static = run_sim(year, 'static-60')
    
    df = pandas.DataFrame({
        'tent' : fn(lens, tent),
        'static-60' : fn(lens, static),
    })
    return df

In [ ]:
one(1929, mk_lens('Bond %')).head()